In [11]:
import ee
import geemap
import xml.etree.ElementTree as ET
import ipywidgets as widgets
import pandas as pd 
from pprint import pprint 

In [12]:
ee.Authenticate() 
ee.Initialize(project="jameswilliamchamberlain")

In [13]:
# UNESCO World Heritage Sites List xml file 
tree = ET.parse("whs.xml")
root = tree.getroot()

data = []

for row in root.findall("row"):
    site = row.findtext('site')
    category = row.findtext('category')
    date_inscribed = row.findtext('date_inscribed')
    region = row.findtext('region')
    url = row.findtext('http_url')
    longitude = row.findtext('longitude')
    latitude = row.findtext('latitude')
    iso = row.findtext('iso_code')
    iso = iso.split(',')
    data.append([site, category, date_inscribed, region, url, longitude, latitude, iso])

df = pd.DataFrame(data, columns=["name", "category", "date inscribed", "region", "url", "longitude", "latitude", "iso"])

df["longitude"] = pd.to_numeric(df["longitude"], errors='coerce')
df["latitude"] = pd.to_numeric(df["latitude"], errors='coerce')

In [14]:
df = df[df['category'] != 'Natural'] 

In [15]:
# df[["longitude", "latitude"]]
df = df.dropna(subset=["longitude", "latitude"]).reset_index(drop=True)

In [16]:
df_disp = df # default 

# regions
iberia = {"es", "pt", "ad"}
balkans = {"bg", "ro", "rs", "hr", "si", "ba", "me", "gr", "al"} 
centeral = {"pl", "de", "he", "ch", "at", "cz", "sk"}
levant = {"il", "jo", "sy", "lb", 'ps'}
fertile_crescent = {"il", "jo", "sy", "lb", 'iq', 'eg', 'ps'}
middle_east = {"il", "jo", "sy", "lb", 'iq', 'eg', 'sa', 'bh', 'qa', 'ae', 'om', 'ye', 'ir', 'tr', 'ps'}
caucasus = {'ge', 'az', 'am'}
stepp_countries = {'tm', 'uz', 'tm', 'kz', 'kg', 'tj'}
indian_subcontinent = {'af', 'pk', 'in', 'np', 'bd', 'lk'}
north_africa = {'ma', 'dz', 'tn', 'ly', 'eg'}
horn_africa = {'er', 'et'}
egypt = {'eg', 'sd', 'il', 'ps'} # old egypt
british_isles = {'gb', 'ie'}

target_iso = {'iq'}
target_url = 'https://whc.unesco.org/en/list/276'
df_disp = df[df['iso'].apply(lambda iso_list: bool(set(iso_list) & target_iso))]
df_disp = df_disp[df_disp['url'] == target_url]
df_disp = df_disp.reset_index(drop=True)

In [17]:
# # Samarra Archaeological City
# target = {"http://whc.unesco.org/en/list/276"}
# df_disp = df[df['url'].apply(lambda url_list: bool(set(url_list) & target))]
# df_disp = df.reset_index(drop=True)

In [18]:
m = geemap.Map()
m.add_points_from_xy(df_disp, x="longitude", y="latitude", layer_name="Sites")

def map_polygon(polygon, collection_name, layer_name, yyyymmdd1="2024-01-01", yyyymmdd2="2024-12-29", num_tasks=10):

    collection = ee.ImageCollection(collection_name) \
        .filterDate(yyyymmdd1, yyyymmdd2) \
        .filterBounds(polygon) \
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20)) \
        .median() \
        .clip(polygon)
    
    vis = {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}

    m.addLayer(collection, vis, layer_name)

def draw_rectangles_for_date(date_val, size_deg=1):

    regions = []

    for _, row in df_disp.iterrows():
        lat = row['latitude']
        lon = row['longitude']

        geometry = ee.Geometry.Rectangle([lon - size_deg/2, lat - size_deg/2, lon + size_deg/2, lat + size_deg/2])
        regions.append(geometry)

    regions_collection = ee.FeatureCollection(regions)

    map_polygon(regions_collection, "COPERNICUS/S2_HARMONIZED", f"S2 {date_val}", f"{date_val}-01-01", f"{date_val}-3-31")

def draw_basic(b):
    dates = [2025, 2020, 2016]
    # dates = [2025, 2024, 2022, 2020, 2018, 2016]
    # dates = [2025, 2024, 2023, 2022, 2021, 2020, 2019, 2018, 20217, 2016]

    for date in dates:
        draw_rectangles_for_date(date)

def draw_selected(b):

    if m.draw_features is None:
        raise ValueError("Error: no roi")
    
    # roi = m.draw_last_feature.geometry()
    roi = ee.FeatureCollection(m.draw_features)

    year_start_val = int(date_slider.value)
    month_start_val = int(month_slider.value)
    time_frame_val = int(time_frame_slider.value)

    month_end_val = month_start_val + time_frame_val
    year_overflow = 0
    if month_end_val > 12:
        month_end_val -= 12
        year_overflow = 1

    yyyymm_start = f"{year_start_val}-{month_start_val}"
    yyyymm_end = f"{year_start_val + year_overflow}-{month_end_val}"
    
    map_polygon(roi, "COPERNICUS/S2_HARMONIZED", yyyymmdd1=yyyymm_start, yyyymmdd2=yyyymm_end, layer_name=f"S2 {yyyymm_start} to {yyyymm_end}")

date_slider = widgets.SelectionSlider(
    options=[str(year) for year in range(2016, 2026)],
    value='2020',
    description='Date:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

month_slider = widgets.SelectionSlider(
    options=[str(month) for month in range(1, 13)],
    value='1',
    description='Month:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

time_frame_slider = widgets.SelectionSlider(
    options=[str(month) for month in range(1, 13)],
    value='3',
    description='timeframe:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

def create_s2_aligned_fishnet(region, tile_size_m=20, chunk_size=500):
    """
        tile region aligned to the S2 projection. 

        TODO: add warning message on run that sometimes tiling fails and that can be seen as a full tile with a single polygon in it, that will need to be-rerun 

        Args:
            region (ee.Geometry):   Region Of Interest (ROI) to tile 
            tile_size_m (int):      Tile size (meters)                                      Default: 50x50m 
            chunk_size (int):       With in tiles generated per chunk (max)                 Default: 500 (note: 1000 is generally safe for 50x50m but has issues on lower resolutions for some reason)

        Returns:
            list:                   List of ee.FeatureCollection tiles
    """
    def name_tile(f):
        
        coords = ee.List(f.geometry().bounds().coordinates().get(0))

        ll = ee.List(coords.get(0))  # lower-left
        ur = ee.List(coords.get(2))  # upper-right

        lon_min = ee.Number(ll.get(0)).multiply(1e6).round().toInt() # .round().multiply(1e6).toInt()
        lat_min = ee.Number(ll.get(1)).multiply(1e6).round().toInt() # .round().multiply(1e6).toInt()
        lon_max = ee.Number(ur.get(0)).multiply(1e6).round().toInt() # .round().multiply(1e6).toInt()
        lat_max = ee.Number(ur.get(1)).multiply(1e6).round().toInt() # .round().multiply(1e6).toInt()

        file_name = ee.String('tile_') \
            .cat(lon_min.format('%06d')).cat('_') \
            .cat(lat_min.format('%06d')).cat('_') \
            .cat(lon_max.format('%06d')).cat('_') \
            .cat(lat_max.format('%06d'))

        return f.set('file_name', file_name)

    def net(sub_region, tile_size_meters):
        """
            Create a fishnet of tiles aligned to the S2 projection. 
            
            This is purely a subfunction to allow for chunking of the region if nessuary else it can cause issues with ee.reduceToVectors and geemap
        """
        # TODO: this could be made simpler by just using the projection 
        # TODO: ... future issue for making this a re-usable function for other projects / projections 
        s2 = ee.ImageCollection("COPERNICUS/S2") \
            .filterBounds(sub_region) \
            .filterDate("2024-01-01", "2024-12-30") \
            .first()

        projection = s2.select("B4").projection()
        px_coords = ee.Image.pixelCoordinates(projection)

        tile_px = tile_size_meters // 10  # 50/10 = 5px by 5px (50mx50m)

        # split 
        tile_ids = px_coords.select("x").divide(tile_px).floor() \
            .multiply(1e6).add(px_coords.select("y").divide(tile_px).floor()) \
            .toInt()
        
        # create a fishnet aligned to S2 projection
        fishnet = tile_ids.reduceToVectors(
            geometry=sub_region,
            geometryType='polygon',
            scale=10,
            bestEffort=True,
            maxPixels=1e13,
        )

        # add relevant ids to each (future file name for each polygon)
        fishnet = fishnet.map(name_tile)

        return fishnet

    # need to fishnet the region into larger chunks to then beable to properly tile else it will fail
    chunks = net(region, tile_size_meters=(tile_size_m * chunk_size)) # chunk_size^2 is total num features in each chunk (100, 500 and 1000 works well (tested))

    fishnet_tiles = []
    for i in range(chunks.size().getInfo()):
        chunk = ee.Feature(chunks.toList(1, i).get(0))
        chunk_tiles = net(chunk.geometry(), tile_size_meters=tile_size_m) 

        # TODO: drop edge tiles 
        fishnet_tiles.append(chunk_tiles)

    return fishnet_tiles

tiles = None

# Tiling 
def create_tiles(b):
    global tiles

    if m.draw_features is None:
        raise ValueError("Error: no roi")

    roi = m.draw_last_feature.geometry()

    fishnet_list = create_s2_aligned_fishnet(roi, tile_size_m=50, chunk_size=100) # TODO: test chunk_size !!!
    
    tiles = fishnet_list


def save_tiles(b, dir="chunk.shp"):
    if tiles is None:
        raise ValueError("Error: no tiles")
    
    for i in range(len(tiles)):
        geemap.ee_to_shp(tiles[i], dir.replace(".shp", f"_{i+1}.shp"))

def show_tiles(b):
    if tiles is None:
        raise ValueError("Error: no tiles")
    
    for i in range(len(tiles)):
        fishnet = ee.FeatureCollection(tiles[i])
        m.addLayer(fishnet, {}, f"Chunk {i+1}")

# Tile Region 
tile_btn = widgets.Button(description="Create Tiles", position="bottomright")
tile_btn.on_click(create_tiles)
save_tiles_btn = widgets.Button(description="Save Tiles", position="bottomright")
save_tiles_btn.on_click(save_tiles)
widget_tiles = widgets.VBox([tile_btn, save_tiles_btn])
m.add_widget(widget_tiles, position="bottomright")

# Show Tiles 
show_tiles_btn = widgets.Button(description="Show Tiles", position="bottomright")
show_tiles_btn.on_click(show_tiles)
m.add_widget(show_tiles_btn, position="bottomright")

# old 
render_btn = widgets.Button(description="Draw Selected Date Layer", position="bottomright") 
draw_btn = widgets.Button(description="Draw Pins Areas 3yrs", position="bottomright")

widget_draw = widgets.VBox([draw_btn, render_btn])

m.add_widget(widget_draw, position="bottomright")

display(widgets.HBox([date_slider, month_slider, time_frame_slider, render_btn]))
render_btn.on_click(draw_selected)
draw_btn.on_click(draw_basic)

m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Samarra Archaeological City

https://whc.unesco.org/en/list/276/documents/   -  Samarra Archaeological City "Maps and Plans" 2007 file 


In [22]:
print(tiles[0].getInfo())

EEException: Collection query aborted after accumulating over 5000 elements.

In [ ]:
# export tasks for tiles with file_name 
import time

def count_active_tasks():
    """
        Counts the number of active tasks in GEE
    """
    tasks = ee.batch.Task.list()
    active_tasks = [task for task in tasks if task.status()['state'] == 'RUNNING']
    return len(active_tasks)

def export_tiles_to_gee(tiles):
    """
        Exports tiles to GEE asset
    """

    # enumerate through each tile and give respective chunk name 
    for i, tile in enumerate(tiles):
        # chunk_name = f"chunk_{i+1}"
        task = ee.batch.Export.table.toAsset(
            collection=tile,
            description=f"chunk_{i+1}",
            # assetId=f"users/jameswilliamchamberlain/tiles/{chunk_name}"
            assetId=f"projects/jameswilliamchamberlain/assets/chunks/chunk_{i+1}"
        )

        task.start()

        active = count_active_tasks()

        while active > 70:
            print(f"Waiting for active tasks to reduce below 70 Active tasks: {active}")
            time.sleep(60)

export_tiles_to_gee(tiles)

NameError: name 'chunk_name' is not defined

In [ ]:
# COPERNICUS/S2_HARMONIZED     
# COPERNICUS/S2
# etc

In [ ]:
# TODO: combine so that I can use tiled system to render quicker as rendering a full country is painfully slow 

# add Iraq full country 
lisb =  ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
# print(lisb.first().getInfo()['properties'])
iraq = lisb.filter(ee.Filter.eq('country_co', 'IZ'))
romania = lisb.filter(ee.Filter.eq('country_co', 'RO'))
# m.addLayer(iraq, {}, "Iraq") # outline Iraq
map_polygon(romania.geometry(), "COPERNICUS/S2_HARMONIZED", "Iraq S2-Haromised 2024", "2024-01-01", "2024-12-31")

# Useful Links 

[Google Earth Engine Editor](https://code.earthengine.google.com/)

[GEE Tasks (Bulk cancel mode)](https://code.earthengine.google.com/tasks)

In [ ]:
# import ee

# ee.Initialize(project="jameswilliamchamberlain")

# import ipygee as ui
# import tqdm  # you can avoid using this if you want

# TM = ui.TaskManager()
# TM

# # Select the tasks you want to cancel, then:

# for task in tqdm.tqdm(TM.selected_tasks()):
#     task.cancel()